In [10]:
# 데이터 불러오기
import pandas as pd
from data_loader import PykrxDataLoader
from pykrx import stock
# import FinanceDataReader as fdr
from statsmodels.regression.linear_model import OLS
import statsmodels.api as sm
import seaborn as sns
import warnings
from sklearn.preprocessing import StandardScaler
from factor_analyzer import FactorAnalyzer
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [30]:
df1 = pd.read_csv("data2/제조업_수익률1.csv")
df2 = pd.read_csv('data2/제조업_시총변화율1.csv')

In [31]:
df1

,회사명,주식코드,연도,총자본증가율(IFRS),유형자산증가율(IFRS),비유동생물자산증가율(IFRS),투자부동산증가율(IFRS),비유동자산증가율(IFRS),유동자산증가율(IFRS),재고자산증가율(IFRS),...,설비투자효율(IFRS),기계투자효율(IFRS),부가가치율(IFRS),노동소득분배율(IFRS),자본분배율(IFRS),이윤분배율(IFRS),날짜,시가,수익율,로그수익률
0,(주)경방,50,2015,-1.22,-4.16,0.0,-1.29,-1.31,0.11,8.61,...,41.80,379.17,35.08,19.66,80.34,20.01,20150504,259500.0,0.500000,-0.405465
1,(주)경방,50,2016,-1.79,-3.70,0.0,-1.30,-1.64,-3.85,-10.48,...,44.22,502.67,35.04,19.01,80.99,28.16,20160502,173000.0,10.849315,-2.472270
2,(주)경방,50,2017,-3.07,-7.75,0.0,-1.26,-2.50,-11.26,-33.22,...,47.66,647.91,37.96,18.25,81.75,23.48,20170502,14600.0,-0.016835,0.016978
3,(주)경방,50,2018,2.29,-6.58,0.0,-1.13,0.96,23.35,58.28,...,41.34,1552.83,31.75,19.89,80.11,32.96,20180502,14850.0,0.320000,-0.277632
4,(주)경방,50,2019,5.68,-69.74,0.0,5.71,-5.79,153.86,-60.76,...,134.74,18933.42,31.82,15.85,84.15,27.55,20190502,11250.0,0.130653,-0.122796
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2671,에스케이스퀘어(주),402340,2022,0.00,0.00,0.0,0.00,0.00,0.00,0.00,...,2127.64,80447.21,80.42,9.21,90.79,91.10,20220502,51600.0,0.214118,-0.194018
2672,에스케이스퀘어(주),402340,2023,-1.77,-1.78,0.0,0.00,-5.59,111.96,0.00,...,-694.84,-33101.32,-98.60,0.00,0.00,0.00,20230502,42500.0,-0.434840,0.570647
2673,에스케이스퀘어(주),402340,2024,-3.32,-28.67,0.0,0.00,-11.28,102.38,0.00,...,-1488.60,-76262.86,-150.41,0.00,0.00,0.00,20240502,75200.0,-0.147392,0.159456
2674,(주)에코프로머티리얼즈,450080,2023,73.92,21.89,0.0,0.00,42.77,117.37,-45.10,...,18.18,28.01,7.31,47.94,52.06,7.24,20230502,NaN,NaN,NaN


In [32]:
# 필요한 컬럼만 추출
df2_subset = df2[['주식코드', '연도', '시가총액']]

# 중복 제거 (필요 시)
df2_subset = df2_subset.drop_duplicates(subset=['주식코드', '연도'])

# 병합
merged_df = pd.merge(df1, df2_subset, on=['주식코드', '연도'], how='left')

In [33]:
# 분위수 기준 그룹 나누기 (시가총액 기준)
merged_df['size_group'] = merged_df.groupby('연도')['시가총액'].transform(
    lambda x: pd.qcut(x, 10, labels=False)
)

# 각 그룹의 평균 수익률 계산
group_mean = merged_df.groupby(['연도', 'size_group'])['로그수익률'].transform('mean')

# SAR 계산
merged_df['SAR'] = merged_df['로그수익률'] - group_mean

In [34]:
merged_df = merged_df.dropna(subset=['SAR'])
# merged_df.to_csv("data/제조업_SAR.csv", index=False)

In [35]:
merged_df.isna().sum()

회사명              0
주식코드             0
연도               0
총자본증가율(IFRS)     0
유형자산증가율(IFRS)    0
                ..
수익율              0
로그수익률            0
시가총액             0
size_group       0
SAR              0
Length: 145, dtype: int64

In [36]:
merged_df

,회사명,주식코드,연도,총자본증가율(IFRS),유형자산증가율(IFRS),비유동생물자산증가율(IFRS),투자부동산증가율(IFRS),비유동자산증가율(IFRS),유동자산증가율(IFRS),재고자산증가율(IFRS),...,노동소득분배율(IFRS),자본분배율(IFRS),이윤분배율(IFRS),날짜,시가,수익율,로그수익률,시가총액,size_group,SAR
0,(주)경방,50,2015,-1.22,-4.16,0.0,-1.29,-1.31,0.11,8.61,...,19.66,80.34,20.01,20150504,259500.0,0.500000,-0.405465,6.744156e+11,3.0,-0.360841
1,(주)경방,50,2016,-1.79,-3.70,0.0,-1.30,-1.64,-3.85,-10.48,...,19.01,80.99,28.16,20160502,173000.0,10.849315,-2.472270,4.715426e+11,2.0,-2.294985
2,(주)경방,50,2017,-3.07,-7.75,0.0,-1.26,-2.50,-11.26,-33.22,...,18.25,81.75,23.48,20170502,14600.0,-0.016835,0.016978,3.961507e+11,1.0,-0.031419
3,(주)경방,50,2018,2.29,-6.58,0.0,-1.13,0.96,23.35,58.28,...,19.89,80.11,32.96,20180502,14850.0,0.320000,-0.277632,4.125998e+11,1.0,-0.131067
4,(주)경방,50,2019,5.68,-69.74,0.0,5.71,-5.79,153.86,-60.76,...,15.85,84.15,27.55,20190502,11250.0,0.130653,-0.122796,3.097926e+11,1.0,0.140489
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2670,(주)에프앤에프,383220,2024,9.19,297.21,0.0,0.00,36.05,-34.33,-11.57,...,11.37,88.63,58.94,20240502,66200.0,-0.056980,0.058668,2.681495e+12,6.0,0.104539
2671,에스케이스퀘어(주),402340,2022,0.00,0.00,0.0,0.00,0.00,0.00,0.00,...,9.21,90.79,91.10,20220502,51600.0,0.214118,-0.194018,7.271433e+12,8.0,-0.037691
2672,에스케이스퀘어(주),402340,2023,-1.77,-1.78,0.0,0.00,-5.59,111.96,0.00,...,0.00,0.00,0.00,20230502,42500.0,-0.434840,0.570647,5.849684e+12,8.0,0.637589
2673,에스케이스퀘어(주),402340,2024,-3.32,-28.67,0.0,0.00,-11.28,102.38,0.00,...,0.00,0.00,0.00,20240502,75200.0,-0.147392,0.159456,1.010625e+13,8.0,0.139720


In [37]:
merged_df = merged_df.iloc[:,3:].drop(['날짜','시가','수익율','로그수익률','시가총액','size_group'],axis=1)

In [38]:
merged_df

,총자본증가율(IFRS),유형자산증가율(IFRS),비유동생물자산증가율(IFRS),투자부동산증가율(IFRS),비유동자산증가율(IFRS),유동자산증가율(IFRS),재고자산증가율(IFRS),자기자본증가율(IFRS),매출액증가율(IFRS),정상영업이익증가율(IFRS),...,기계장비율(IFRS),자본집약도(IFRS),총자본투자효율(IFRS),설비투자효율(IFRS),기계투자효율(IFRS),부가가치율(IFRS),노동소득분배율(IFRS),자본분배율(IFRS),이윤분배율(IFRS),SAR
0,-1.22,-4.16,0.0,-1.29,-1.31,0.11,8.61,3.10,19.75,20.36,...,46.29,2047.99,8.57,41.80,379.17,35.08,19.66,80.34,20.01,-0.360841
1,-1.79,-3.70,0.0,-1.30,-1.64,-3.85,-10.48,2.75,1.99,10.22,...,0.00,0.00,8.89,44.22,502.67,35.04,19.01,80.99,28.16,-2.294985
2,-3.07,-7.75,0.0,-1.26,-2.50,-11.26,-33.22,3.00,-8.21,-7.44,...,0.00,0.00,9.12,47.66,647.91,37.96,18.25,81.75,23.48,-0.031419
3,2.29,-6.58,0.0,-1.13,0.96,23.35,58.28,2.56,-3.13,-1.11,...,0.00,0.00,7.22,41.34,1552.83,31.75,19.89,80.11,32.96,-0.131067
4,5.68,-69.74,0.0,5.71,-5.79,153.86,-60.76,1.58,-9.15,-0.01,...,0.00,0.00,6.22,134.74,18933.42,31.82,15.85,84.15,27.55,0.140489
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2670,9.19,297.21,0.0,0.00,36.05,-34.33,-11.57,17.77,-10.37,-23.65,...,0.00,0.00,28.54,140.78,0.00,36.16,11.37,88.63,58.94,0.104539
2671,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,6.34,2127.64,80447.21,80.42,9.21,90.79,91.10,-0.037691
2672,-1.77,-1.78,0.0,0.00,-5.59,111.96,0.00,-7.60,-69.96,-81.79,...,5.57,77491.44,-2.38,-694.84,-33101.32,-98.60,0.00,0.00,0.00,0.637589
2673,-3.32,-28.67,0.0,0.00,-11.28,102.38,0.00,-4.23,-0.07,19.71,...,0.00,0.00,-3.75,-1488.60,-76262.86,-150.41,0.00,0.00,0.00,0.139720


In [39]:
merged_df.to_csv("data/제조업_SAR.csv", index=False)